In [40]:
from bnm import BNMetrics, generate_random_dag, dag_to_cpdag, generate_synthetic_data_from_dag
import networkx as nx
import pandas as pd
import numpy as np
from castle.algorithms import PC

## DAG Comparison: Motivation and Setup

Let’s assume we have two Directed Acyclic Graphs (DAGs) and want to compare them.  
There are several important reasons to perform DAG comparisons:

- **Condition-based Comparison**: Compare DAGs estimated from different conditions (e.g., time points, regions, or experimental groups) to:
  - Identify persistent relationships.
  - Detect structural changes across scenarios.
- **Validation**: Compare a DAG learned by a causal discovery algorithm against a known true DAG.


To illustrate these use cases, we will explore two scenarios:

1. **Case 1**: Simulate two systems, each comprising 100 attributes, generate corresponding datasets, estimate DAGs, and compare them.
2. **Case 2**: Simulate a single *true* system, generate data from it, learn a DAG using a structure learning algorithm, and compare the estimated DAG to the true one.

> **Note**: In this notebook, the terms *attributes*, *nodes*, and *variables* are used interchangeably.


## Case 1: Condition-Based Comparison of One System

We simulate two different DAGs that represent the same underlying system observed under different conditions  
(e.g., different time points, regions, etc.).

- First, we create two DAGs.  
- Then, we compare their complexity and potential causal relationships using descriptive metrics.  
- Next, we assess the similarity between the two DAGs using comparison metrics.  
- Finally, we visually inspect the Markov blankets of selected nodes to explore local structural differences.


### Generate Two Random DAGs

We generate two synthetic DAGs using the `generate_random_dag` function.  
Each DAG contains 100 nodes.

- The first DAG is generated with an edge probability of 3%.  
- The second DAG is generated with an edge probability of 2%.  

A random seed is set to ensure the results are reproducible.

In [41]:
system1 = generate_random_dag(n_nodes=100, edge_prob=0.03, seed=55)
system2 = generate_random_dag(n_nodes=100, edge_prob=0.02, seed=66)

### Compare Complexity

Next, we compute a set of descriptive metrics for both DAGs.  
To do this, we instantiate a `BNMetrics` object and call its `compare_df` method.

At this stage, we focus on calculating descriptive metrics for the global structure to understand the differences in complexity between the two systems.


In [42]:
bnm_obj = BNMetrics(system1, system2)
np.transpose(
    bnm_obj.compare_df(descriptive_metrics=['n_edges', 'n_colliders', 'n_isolated_nodes'], comparison_metrics=None).query("node_name == 'All'")
).reset_index()[1:].rename(columns={'index': 'metric', 0: 'value'})

,metric,value
1,n_edges_base,148.0
2,n_edges,99.0
3,n_colliders_base,137.0
4,n_colliders,63.0
5,n_isolated_nodes_base,5.0
6,n_isolated_nodes,16.0


The `_base` postfix in the metric names indicates that the metric was calculated for the first DAG (the one in the first position of the `BNMetrics` object).  
Metrics without the `_base` postfix refer to the second DAG (the one in the second position).

From the descriptive metrics, we observe that the first system is more complex:

- The number of edges in the first system is 148, compared to 99 in the second system.  
- Similarly, the number of colliders is 137 in the first system versus 63 in the second.  
- Complexity is further reflected in the number of isolated nodes: the second system has 16, while the first has only 5.

These metrics provide an initial understanding of the structural differences between the two systems.


### Similarity

Next, we focus on assessing the similarity between the two systems.  
At this stage, we are interested in similarity metrics for the global structures only.

To calculate these metrics, we again use the `compare_df` method.  
This time, we set `descriptive_metrics=None` and `comparison_metrics='All'` to compute only the comparison metrics.


In [43]:
np.transpose(
    bnm_obj.compare_df(descriptive_metrics=None, comparison_metrics='All').query("node_name == 'All'")
)[1:].reset_index().rename(columns={'index': 'metric', 0: 'value'})

,metric,value
0,additions,94
1,deletions,143
2,reversals,3
3,shd,240
4,hd,237
5,tp,2
6,fp,97
7,fn,146
8,precision,0.020202
9,recall,0.013514


The comparison metrics reveal the degree of similarity between the two DAGs.

To transform one DAG into the other, the following changes are required:
- 94 additions  
- 143 deletions  
- 3 reversals  

The **Structural Hamming Distance (SHD)**—the total number of additions, deletions, and reversals—is 240.  
The **Hamming Distance (HD)**, which excludes reversals, is 237.

Only two edges are shared between the two DAGs.  
There are 97 **false positives**, corresponding to the number of required additions and reversals.  
There are 146 **false negatives**, corresponding to the number of required deletions and reversals.

- **Precision** (the proportion of correct edges in the second DAG) is approximately 2%.  
- **Recall** (the proportion of correct edges from the first DAG recovered in the second) is approximately 1%.

These results are expected, given that the first DAG is more complex.


Now we want to identify which Markov blankets contain at least one true positive edge.  
This is straightforward — we simply filter the comparison DataFrame to retain only the rows where the number of true positives is greater than 0.

In [44]:
bnm_obj.compare_df(descriptive_metrics=None, comparison_metrics='All').query("tp > 0")[['node_name', 'tp']]

,node_name,tp
0,All,2
46,X_39,1
49,X_65,1
68,X_89,1
97,X_46,1


The Markov blankets of nodes `X_39`, `X_65`, `X_89`, and `X_46` each contain a matching edge.  
In total, there are only two matching edges between the two DAGs, yet we observe four nodes with true positives in their Markov blankets. Why?

This is because an edge between two nodes will appear in the Markov blanket of both nodes involved.  
In other words, an edge from node A to node B will be present in the Markov blanket of both A and B.



### Visual inspection of nodes of interest
We will now visualize the Markov blankets of the nodes of interest — those that contain matching edges — in a neat, side-by-side format.

We start with node `X_39`.



In [45]:
bnm_obj.compare_two_bn(nodes=['X_39'], name1='DAG1 - MB of node X_39', name2='DAG1 - MB of node X_39')

In the plot above, we see the two Markov blankets of node `X_39`.  
The matching edge from `X_89` to `X_39` is highlighted in red, and the node of interest (`X_39`) is highlighted in green.

- The Markov blanket of `X_39` in the first DAG contains 3 directed edges.  
- In the second DAG, it contains 4 directed edges.

The first DAG includes more colliders — 3 compared to 2 in the second DAG.

To transform the first DAG's Markov blanket into the second's, we would need to make 3 additions and 2 deletions.

In terms of edge agreement:
- 25% of the second DAG's edges match those in the first.  
- 33% of the first DAG's edges match those in the second.


In [46]:
bnm_obj.compare_df(descriptive_metrics='All', comparison_metrics='All').query("node_name == 'X_39'")

,node_name,n_edges_base,n_edges,n_nodes_base,n_nodes,n_colliders_base,n_colliders,n_root_nodes_base,n_root_nodes,n_leaf_nodes_base,n_leaf_nodes,n_isolated_nodes_base,n_isolated_nodes,n_directed_arcs_base,n_directed_arcs,n_undirected_arcs_base,n_undirected_arcs,n_reversible_arcs_base,n_reversible_arcs,n_in_degree_base,n_in_degree,n_out_degree_base,n_out_degree,additions,deletions,reversals,shd,hd,tp,fp,fn,precision,recall,f1_score
46,X_39,3.0,4.0,4.0,5.0,3.0,2.0,3.0,3.0,1.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3,2,0,5,5,1,3,2,0.25,0.333333,0.285714


Now let's move on to the Markov blankets of the node `X_89`.

In [47]:
bnm_obj.compare_two_bn(nodes=['X_89'])

In the plot above, we see the two Markov blankets of node `X_89`.  
The matching edge from `X_89` to `X_39` is highlighted in red, and the node of interest (`X_89`) is highlighted in green.

- The Markov blanket of `X_89` in the first DAG contains 13 directed edges.  
- In the second DAG, it contains 9 directed edges.

The first DAG includes more colliders — 12 compared to 7 in the second DAG.

To transform the first DAG's Markov blanket into the second's, we would need to make 8 additions and 12 deletions.

In terms of edge agreement:
- 11% of the second DAG's edges match those in the first.  
- 8% of the first DAG's edges match those in the second.


In [48]:
bnm_obj.compare_df(descriptive_metrics='All', comparison_metrics='All').query("node_name == 'X_89'")

,node_name,n_edges_base,n_edges,n_nodes_base,n_nodes,n_colliders_base,n_colliders,n_root_nodes_base,n_root_nodes,n_leaf_nodes_base,n_leaf_nodes,n_isolated_nodes_base,n_isolated_nodes,n_directed_arcs_base,n_directed_arcs,n_undirected_arcs_base,n_undirected_arcs,n_reversible_arcs_base,n_reversible_arcs,n_in_degree_base,n_in_degree,n_out_degree_base,n_out_degree,additions,deletions,reversals,shd,hd,tp,fp,fn,precision,recall,f1_score
68,X_89,13.0,9.0,12.0,9.0,12.0,7.0,6.0,5.0,4.0,3.0,0.0,0.0,13.0,9.0,0.0,0.0,3.0,1.0,1.0,0.0,4.0,3.0,8,12,0,20,20,1,8,12,0.111111,0.076923,0.090909


Node `X_65`

In [49]:
bnm_obj.compare_two_bn(nodes=['X_65'])

In the plot above, we see the two Markov blankets of node `X_65`.  
The matching edge from `X_46` to `X_65` is highlighted in red, and the node of interest (`X_65`) is highlighted in green.

- The Markov blanket of `X_65` in the first DAG contains 14 directed edges.  
- In the second DAG, it contains just 1 directed edge.

The first DAG includes 20 colliders, whereas the second DAG does not include any.

To transform the first DAG's Markov blanket into the second's, we would need to delete 13 edges.

In terms of edge agreement:
- 100% of the second DAG's edges match those in the first.  
- 7% of the first DAG's edges match those in the second.


In [50]:
bnm_obj.compare_df(descriptive_metrics='All', comparison_metrics='All').query("node_name == 'X_65'")

,node_name,n_edges_base,n_edges,n_nodes_base,n_nodes,n_colliders_base,n_colliders,n_root_nodes_base,n_root_nodes,n_leaf_nodes_base,n_leaf_nodes,n_isolated_nodes_base,n_isolated_nodes,n_directed_arcs_base,n_directed_arcs,n_undirected_arcs_base,n_undirected_arcs,n_reversible_arcs_base,n_reversible_arcs,n_in_degree_base,n_in_degree,n_out_degree_base,n_out_degree,additions,deletions,reversals,shd,hd,tp,fp,fn,precision,recall,f1_score
49,X_65,14.0,1.0,15.0,2.0,20.0,0.0,11.0,1.0,3.0,1.0,0.0,0.0,14.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,3.0,0.0,0,13,0,13,13,1,0,13,1.0,0.071429,0.133333


Node `X_46`

In [51]:
bnm_obj.compare_two_bn(nodes=['X_46'])

In the plot above, we see the two Markov blankets of node `X_46`.  
The matching edge from `X_46` to `X_65` is highlighted in red, and the node of interest (`X_46`) is highlighted in green.

- The Markov blanket of `X_46` in the first DAG contains 11 directed edges.  
- In the second DAG, it contains 6 directed edges.

The first DAG includes more colliders — 15 compared to 6 in the second DAG.

To transform the first DAG's Markov blanket into the second's, we would need to make 5 additions and 10 deletions.

In terms of edge agreement:
- 17% of the second DAG's edges match those in the first.  
- 9% of the first DAG's edges match those in the second.


In [52]:
bnm_obj.compare_df(descriptive_metrics='All', comparison_metrics='All').query("node_name == 'X_46'")

,node_name,n_edges_base,n_edges,n_nodes_base,n_nodes,n_colliders_base,n_colliders,n_root_nodes_base,n_root_nodes,n_leaf_nodes_base,n_leaf_nodes,n_isolated_nodes_base,n_isolated_nodes,n_directed_arcs_base,n_directed_arcs,n_undirected_arcs_base,n_undirected_arcs,n_reversible_arcs_base,n_reversible_arcs,n_in_degree_base,n_in_degree,n_out_degree_base,n_out_degree,additions,deletions,reversals,shd,hd,tp,fp,fn,precision,recall,f1_score
97,X_46,11.0,6.0,12.0,7.0,15.0,6.0,9.0,4.0,2.0,2.0,0.0,0.0,11.0,6.0,0.0,0.0,0.0,2.0,3.0,1.0,2.0,2.0,5,10,0,15,15,1,5,10,0.166667,0.090909,0.117647


## Case 2: Validation of Learned DAG

We simulate a true DAG, generate a dataset based on it, and then apply a structure learning algorithm to estimate a DAG from the data.  
Finally, we compare the true DAG with the learned DAG.

- First, we estimate the DAG from data using a structure learning algorithm.  
- Then, we compare the complexity and potential causal relationships between the true DAG and the learned DAG using descriptive metrics.  
- Next, we assess the similarity between the two DAGs using comparison metrics.  
- Finally, we visually inspect the Markov blankets of selected nodes to explore local structural differences.


### Create Learned DAG

We generate one synthetic DAG using the `generate_random_dag` function.  
The DAG contains 40 nodes with an edge probability of 10%.  
A random seed is set to ensure the results are reproducible.


In [53]:
true_dag = generate_random_dag(n_nodes=40, edge_prob=0.1, seed=77)

We then simulate data using the `generate_synthetic_data_from_dag` function.  
The data generation assumes a linear relationship between variables, with continuous data types and standard normal distribution for the noise terms.

After the simulation, we reorder the column names of the generated dataset to match the order of nodes in the underlying DAG, ensuring consistency between the structure and the data for further comparison.


In [54]:
data = generate_synthetic_data_from_dag(true_dag, n_samples=1000, stdev=1.0, seed=55)
data = data[list(true_dag.nodes)]

We employ the PC algorithm (stable version) from the `gCastle` package with an alpha parameter of 0.05.  
The output of the algorithm is then converted to an adjacency matrix in `np.array` format.

In [55]:
pc = PC(variant='stable', alpha=0.05)
pc.learn(data)
learned_dag = np.array(pc._causal_matrix)

### Compare Complexity

We compute a set of descriptive metrics for both the true and the learned DAGs.  
To do this, we instantiate a `BNMetrics` object and call its `compare_df` method.  

Since the PC algorithm outputs a CPDAG, we convert the true DAG into its CPDAG form to ensure that the true and learned structures are comparable.

At this stage, we focus on descriptive metrics for the global structure to understand the differences in complexity between the true and learned CPDAGs.


In [56]:
bnm_obj_2 = BNMetrics(dag_to_cpdag(true_dag), learned_dag)
np.transpose(
    bnm_obj_2.compare_df(descriptive_metrics=['n_edges', 'n_colliders', 'n_isolated_nodes'], comparison_metrics=None).query("node_name == 'All'")
)[1:].reset_index().rename(columns={'index': 'metric', 0: 'value'})

,metric,value
0,n_edges_base,78.0
1,n_edges,60.0
2,n_colliders_base,82.0
3,n_colliders,46.0
4,n_isolated_nodes_base,0.0
5,n_isolated_nodes,0.0


From the descriptive metrics, we observe that the true CPDAG is more complex:

- The number of edges in the true CPDAG is 78, compared to 60 in the learned CPDAG.
- Similarly, the number of colliders is 82 in the true CPDAG versus 46 in the learned CPDAG.
- There are no isolated nodes in either the true or the learned CPDAG.

These metrics provide an initial understanding of the structural differences between the two CPDAGs.


### Similarity

Next, we assess the similarity between the two CPDAGs.  
At this point, our focus remains on similarity metrics for the global structures.

To calculate these metrics, we again use the `compare_df` method.  
This time, we set `descriptive_metrics=None` and `comparison_metrics='All'` to compute only the comparison metrics.


In [57]:
np.transpose(
    bnm_obj_2.compare_df(descriptive_metrics=None, comparison_metrics='All').query("node_name == 'All'")
)[1:].reset_index().rename(columns={'index': 'metric', 0: 'value'})

,metric,value
0,additions,9
1,deletions,27
2,reversals,18
3,shd,54
4,hd,36
5,tp,33
6,fp,27
7,fn,45
8,precision,0.55
9,recall,0.423077


The comparison metrics reveal the similarity between the two CPDAGs.

To transform the true CPDAG into the learned CPDAG, the following operations are required:  
- 9 additions  
- 27 deletions  
- 18 reversals  

The Structural Hamming Distance (SHD)—defined as the sum of additions, deletions, and reversals—is 54.  
The Hamming Distance (HD), which excludes reversals, is 36.

There are 33 matching edges between the two CPDAGs.  
The number of false positives is 27, corresponding to the required additions and reversals.  
The number of false negatives is 45, corresponding to the required deletions and reversals.

- Precision, defined as the proportion of correctly predicted edges out of all predicted edges, is approximately 55%.  
- Recall, defined as the proportion of correctly predicted edges out of all true edges, is approximately 42%.  
- The F1 score, which is the harmonic mean of precision and recall, is approximately 0.478.


### Visual Inspection of Nodes of Interest

Next, we want to visualize the Markov blankets of nodes with the highest F1 scores.  
This helps us explore which parts of the learned CPDAG most closely resemble the true structure at a local level.


In [58]:
highest_f1 = bnm_obj_2.compare_df(descriptive_metrics=None, comparison_metrics='All').query("f1_score == 1").node_name.values
bnm_obj_2.compare_two_bn(nodes=highest_f1, name1='True DAG', name2='Learned DAG')

The Markov blankets of these five nodes match between the true and learned DAGs. This means that the algorithm was able to estimate them correctly.

Next, we want to focus on the second node with the highest F1 score (excluding perfect matches).  
To do this, we filter the comparison DataFrame to retain only rows where the F1 score is greater than 0.5 and less than 1.


In [59]:
bnm_obj_2.compare_df(descriptive_metrics=None, comparison_metrics='All').query("f1_score < 1 & f1_score > 0.5")

,node_name,additions,deletions,reversals,shd,hd,tp,fp,fn,precision,recall,f1_score
22,X_22,2,3,0,5,5,5,2,3,0.714286,0.625,0.666667
32,X_18,0,4,0,4,4,3,0,4,1.0,0.428571,0.6
35,X_16,3,1,0,4,4,5,3,1,0.625,0.833333,0.714286


From the table above, we see that the second-highest F1 score belongs to node `X_16`.

In [60]:
bnm_obj_2.compare_two_bn(nodes=['X_16'], name1='True DAG', name2='Learned DAG')

In the plot above, we see the two Markov blankets of node `X_16`, with 5 matching edges highlighted in red.  
The node of interest (`X_16`) is highlighted in green.

- The Markov blanket of `X_16` in the true CPDAG contains 6 directed edges.  
- In the learned CPDAG, it contains 8 directed edges.

The true CPDAG includes fewer colliders — 7 compared to 9 in the learned CPDAG.

To transform the true CPDAG's Markov blanket into the learned one, we would need to make 3 additions and 1 deletion.

In terms of accuracy:
- 62% of all predicted edges are correct (precision).  
- 83% of all true edges are correctly predicted (recall).  
- The F1 score is 71%.


In [61]:
bnm_obj_2.compare_df(descriptive_metrics='All', comparison_metrics='All').query("node_name == 'X_16'")

,node_name,n_edges_base,n_edges,n_nodes_base,n_nodes,n_colliders_base,n_colliders,n_root_nodes_base,n_root_nodes,n_leaf_nodes_base,n_leaf_nodes,n_isolated_nodes_base,n_isolated_nodes,n_directed_arcs_base,n_directed_arcs,n_undirected_arcs_base,n_undirected_arcs,n_reversible_arcs_base,n_reversible_arcs,n_in_degree_base,n_in_degree,n_out_degree_base,n_out_degree,additions,deletions,reversals,shd,hd,tp,fp,fn,precision,recall,f1_score
35,X_16,6.0,8.0,7.0,9.0,7.0,9.0,5.0,6.0,1.0,2.0,0.0,0.0,6.0,8.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,2.0,3,1,0,4,4,5,3,1,0.625,0.833333,0.714286


Next, we are interested in visualizing Markov blankets that contain no matching edges.  
This allows us to investigate local structures where the learned DAG completely different from the true DAG.


In [62]:
lowest_f1 = bnm_obj_2.compare_df(descriptive_metrics=None, comparison_metrics='All').query("f1_score == 0").node_name.values
bnm_obj_2.compare_two_bn(nodes=lowest_f1, name1='True DAG', name2='Learned DAG')

## Conclusions

This notebook demonstrated two key use cases for comparing Directed Acyclic Graphs (DAGs) using the `BNMetrics` package: condition-based comparison and validation of a learned DAG.

In **Case 1 (Condition-Based Comparison)**, we simulated two DAGs representing the same system under different conditions.  
Descriptive metrics revealed significant differences in complexity, with one system being more densely connected.  
Similarity metrics showed low overlap between the two structures, which was further confirmed through Markov blanket analysis — only a few nodes shared common edges across conditions.

In **Case 2 (Validation of Learned DAG)**, we evaluated how accurately a structure learning algorithm could reconstruct a known true DAG from simulated data.  
Global comparison metrics indicated a moderate match between the true and learned structures.  
At the local level, Markov blanket analysis revealed that some were estimated very accurately (with high F1 scores), while others showed no overlap at all.

These examples highlight the value of analyzing both global structures and individual local structures to gain a deeper understanding of the similarities and differences between DAGs.
